In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import threading

c:\Users\user\anaconda3\envs\ML_temp\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\user\anaconda3\envs\ML_temp\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\user\anaconda3\envs\ML_temp\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\user\anaconda3\envs\ML_temp\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\user\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
# Load a model
model = YOLO("yolo11n-pose.pt")  # load an official model
# 전역 변수
frame = None  # 웹캠에서 캡처한 프레임
captured_frames = []  # 캡처된 프레임 저장
lock = threading.Lock()

# 키포인트 인덱스
KEYPOINTS_TO_CHECK = [0, 5, 6, 7, 8, 9, 10]  # 머리(0), 왼쪽 어깨(5), 오른쪽 어깨(6), 팔/다리(7-10)

# Predict with the model
results = model("https://ultralytics.com/images/bus.jpg")

  0%|          | 0.00/5.97M [00:00<?, ?B/s]

  0%|          | 0.00/134k [00:00<?, ?B/s]

image 1/1 c:\AI_class_yang\AI_code\  \body_part_measurement_source\backend\bus.jpg: 640x480 4 persons, 18.0ms
Speed: 7.0ms preprocess, 18.0ms inference, 35.0ms postprocess per image at shape (1, 3, 640, 480)


In [ ]:
# 웹캠 캡처 함수
def capture_webcam():
    global frame
    cap = cv2.VideoCapture(0)  # 0번 웹캠
    while True:
        ret, img = cap.read()
        if not ret:
            break
        with lock:
            frame = img
    cap.release()

# YOLO Pose로 사람 감지 및 조건부 캡처
def process_frame():
    global frame, captured_frames
    while True:
        if frame is None:
            continue

        with lock:
            img = frame.copy()

        # YOLO Pose 모델 실행
        results = model(img)
        for result in results:
            keypoints = result.keypoints.numpy() if result.keypoints is not None else []

            # 키포인트 조건 확인 (머리, 팔, 다리 모두 감지 여부)
            if check_keypoints_complete(keypoints):
                # 조건 충족 시 캡처
                with lock:
                    captured_frames.append(img)

# 키포인트 검증 함수
def check_keypoints_complete(keypoints):
    """
    특정 키포인트들이 모두 감지되었는지 확인.
    - keypoints: [[x, y, confidence], ...] 형태의 배열
    """
    for idx in KEYPOINTS_TO_CHECK:
        if idx >= len(keypoints) or keypoints[idx][2] < 0.5:  # 키포인트 confidence < 0.5는 감지 실패로 간주
            return False
    return True


In [8]:
results[0].show()